In [1]:
!pip install openpyxl
!jupyter trust Mapping_Ridership.ipynb
!pip install branca==0.3.1 

Signing notebook: Mapping_Ridership.ipynb


In [2]:
import numpy as np
import pandas as pd
import folium
from project_tools.utils import calc_stn_perc_diffs, add_markers_to_map
pd.set_option('display.max_columns', None)

## Import data

In [3]:
stn_names = pd.read_excel('Data/station-names.xls')
jan2019 = pd.read_excel('Data/ridership_2019/Ridership_201901.xlsx', header=1, index_col=0)
jan2022 = pd.read_excel('Data/ridership_2022/Ridership_202201.xlsx', header=1, index_col=0)
jan2022.head()

,RM,EN,EP,NB,BK,AS,MA,19,12,LM,FV,CL,SL,BF,HY,SH,UC,FM,CN,PH,WC,LF,OR,RR,OW,EM,MT,PL,CC,16,24,GP,BP,DC,CM,CV,ED,NC,WP,SS,SB,SO,MB,WD,OA,WS,ML,BE,PC,AN,Exits
RM,9.333333,50.190476,41.619048,27.000000,140.904762,37.619048,56.047619,40.571429,44.000000,11.904762,40.571429,25.047619,16.142857,7.809524,13.952381,7.095238,6.190476,5.904762,9.619048,3.142857,10.380952,4.666667,3.380952,8.952381,14.047619,128.047619,97.523810,173.761905,102.285714,91.428571,87.476190,20.523810,26.285714,17.666667,9.571429,2.619048,5.238095,1.619048,5.285714,4.857143,9.190476,15.523810,7.809524,2.047619,4.476190,7.000000,3.190476,4.142857,5.095238,6.523810,1475.285714
EN,57.666667,12.809524,40.904762,30.857143,227.333333,41.047619,81.714286,82.619048,112.238095,25.047619,47.523810,34.095238,20.000000,16.047619,19.952381,9.714286,13.666667,12.857143,10.809524,5.047619,13.142857,3.285714,5.380952,11.761905,19.190476,253.761905,196.571429,287.476190,218.047619,96.000000,84.857143,26.857143,31.619048,24.761905,10.190476,2.761905,7.904762,3.952381,5.285714,7.380952,12.571429,43.666667,19.285714,2.000000,13.714286,9.809524,5.333333,6.476190,3.095238,4.619048,2332.714286
EP,45.428571,34.095238,6.761905,22.523810,170.333333,23.714286,42.857143,44.761905,56.476190,14.952381,24.761905,10.238095,14.619048,6.904762,7.095238,5.190476,4.714286,6.142857,6.428571,2.238095,5.952381,3.142857,2.380952,12.095238,4.238095,95.571429,94.714286,89.476190,86.428571,31.285714,29.523810,10.000000,8.000000,6.619048,1.000000,1.619048,3.238095,1.095238,4.142857,3.904762,2.142857,19.714286,4.952381,2.285714,5.809524,4.761905,2.714286,5.142857,1.761905,2.476190,1096.428571
NB,28.619048,26.285714,24.000000,5.476190,64.714286,16.714286,27.428571,35.571429,33.333333,10.619048,32.571429,11.714286,10.857143,7.333333,5.714286,2.333333,4.428571,5.380952,5.047619,5.857143,5.238095,1.809524,3.476190,4.190476,5.142857,69.761905,72.714286,72.333333,80.904762,33.523810,22.809524,7.047619,12.333333,6.380952,2.952381,3.380952,3.714286,1.095238,3.619048,2.571429,1.761905,27.285714,3.523810,1.333333,8.095238,3.523810,1.857143,3.142857,0.714286,2.523810,832.761905
BK,153.285714,238.190476,183.428571,68.428571,19.523810,86.047619,96.238095,105.095238,110.809524,41.190476,93.285714,45.380952,49.952381,29.523810,28.857143,17.619048,25.285714,21.904762,22.571429,21.952381,24.380952,12.809524,10.095238,9.714286,28.380952,139.666667,106.000000,153.333333,130.523810,70.142857,63.761905,26.952381,22.380952,17.571429,4.142857,8.904762,17.238095,6.857143,15.761905,5.428571,7.904762,73.190476,22.714286,5.428571,35.285714,13.380952,14.238095,22.047619,5.190476,14.904762,2546.904762


## Mapping

### San Francisco

#### Calculate total percent difference between 2019 and 2022 for each station

In [4]:
SF_stns = ['EM', 'MT', 'PL', 'CC']
SF_percent_diffs = calc_stn_perc_diffs(jan2019, jan2022, SF_stns)

#### Plot percent differences on map

Code source: https://www.python-graph-gallery.com/313-bubble-map-with-folium

In [5]:
# Make an empty map

SF_map = folium.Map(location=[37.78656375,-122.4051605], tiles="cartodbpositron", zoom_start=15)

In [6]:
# Make a data frame with dots to show on the map

SF_mapping_df = pd.DataFrame({
   'lon':[-122.3970, -122.401942, -122.4080, -122.4137],
   'lat':[37.7929, 37.789355, 37.7844, 37.7796],
   'name':['Embarcadero: '+str(round(SF_percent_diffs[0]))+'%', 
           'Montgomery Street: '+str(round(SF_percent_diffs[1]))+'%', 
           'Powell Street: '+str(round(SF_percent_diffs[2]))+'%', 
           'Civic Center: '+str(round(SF_percent_diffs[3]))+'%'],
   'value': SF_percent_diffs
}, dtype=str)

SF_mapping_df

,lon,lat,name,value
EM,-122.397,37.7929,Embarcadero: -86%,-86.17467169727105
MT,-122.401942,37.789355,Montgomery Street: -88%,-88.09026368383158
PL,-122.408,37.7844,Powell Street: -75%,-74.75105067323393
CC,-122.4137,37.7796,Civic Center: -78%,-77.63420614311872


In [7]:
# Add markers for ridership percent change to the map

add_markers_to_map(SF_mapping_df, 2, 'crimson', SF_map)

In [8]:
# Save map file to figures/

SF_map.save('figures/SF_interactive_map.html')

### Berkeley

In [9]:
berk_stns = ['BK', 'NB']
berk_percent_diffs = calc_stn_perc_diffs(jan2019, jan2022, berk_stns)

In [10]:
# Make an empty map

berk_map = folium.Map(location=[37.8715, -122.2730], tiles="cartodbpositron", zoom_start=15)

In [11]:
# Make a data frame with dots to show on the map

berk_mapping_df = pd.DataFrame({
   'lon':[-122.2681, -122.2834],
   'lat':[37.8701, 37.8740],
   'name':['Downtown Berkeley: '+str(round(berk_percent_diffs[0]))+'%', 
           'North Berkeley: '+str(round(berk_percent_diffs[1]))+'%'],
   'value': berk_percent_diffs}, dtype=str)

berk_mapping_df

,lon,lat,name,value
BK,-122.2681,37.8701,Downtown Berkeley: -75%,-74.59268290225398
NB,-122.2834,37.874,North Berkeley: -80%,-79.80586818123625


In [12]:
# Add markers for ridership percent change to the map

add_markers_to_map(berk_mapping_df, 6, 'crimson', berk_map)

In [13]:
# Save map file to figures/

berk_map.save('figures/berkeley_interactive_map.html')